#### Llama2 4bit 양자화 파인튜닝
1. ##### 4비트 양자화 QLoRA 파인튜닝 (효율성) * 파라미터를 고정 시키고 추가데이터만 튜닝

2. python module 설치
   * %pip install torch fairscale fire sentencepiece transformers protobuf accelerate peft bitsandbytes trl\
   * %pip install accelerate==0.26.1 peft==0.8.2 bitsandbytes==0.42.0 transformers==4.37.2 trl==0.7.10 \

In [ ]:
%pip install torch fairscale fire sentencepiece transformers protobuf accelerate peft bitsandbytes trl

In [15]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

import json
import jsonlines
import os

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [2]:
from datasets import Dataset
from datasets import load_from_disk

#### 학습데이터셋 불러오기

In [3]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

##### 라바 허깅페이스 모델 불러오기

In [4]:
model_id = "/data/bwllm/models/casllm-base-7b-hf"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

##### 토큰라이저 불러오기

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


##### PEFT 파라미터 설정
> Parameter-Efficient Fine-Tuning (PEFT)은 모델 파라미터의 작은 하위 집합만 업데이트

In [6]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

##### 훈련 파라메터 설정

In [11]:
output_peft_model = "/data/bwllm/models/audit_sllm7b_peft"
training_params = TrainingArguments(
    output_dir=output_peft_model,
    num_train_epochs=5,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2, # 더 큰 배치 효과를 위해 증가 (1 -> 2)
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,   # True 인 것을 메모리 오버플로우 방지를 위해 True 로 변경
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

##### 훈련설정

In [12]:
train_dataset_folder = "/data/bwllm/dataset/alpaca_en_dataset.jsonl"
dataset = load_from_disk(train_dataset_folder)

# 데이터 확인
dataset[:5]

{'dataset': ['<s>[INST] Give three tips for staying healthy. [/INST] [REF]  [/REF] 1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule. </s>',
  '<s>[INST] What are the three primary colors? [/INST] [REF]  [/REF] The three primary colors are red, blue, and yellow. </s>',
  '<s>[INST] Describe the structure of an atom. [/INST] [REF]  [/REF] An atom is made up of a nucleus, which contains protons and neutrons, surrounded by electrons that travel in orbits around the nucleus. The protons and neutrons have a positive charge, while the electrons have a negative charge, resulting in an overall neutral atom. The number of each particle determines the atomic number and the type of atom. </s>',
  '<s>[INST] How can we reduce air pollution? [/INST] [REF]  [/REF] There are a number of ways to reduce air pollution, such as shifting to renewable ene

In [13]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="dataset",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/data/bwllm/venv/llama/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:225: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [14]:
trainer.train()

OutOfMemoryError: Caught OutOfMemoryError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/data/bwllm/venv/llama/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py", line 83, in _worker
    output = module(*input, **kwargs)
  File "/data/bwllm/venv/llama/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1511, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/data/bwllm/venv/llama/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1520, in _call_impl
    return forward_call(*args, **kwargs)
  File "/data/bwllm/venv/llama/lib/python3.10/site-packages/peft/peft_model.py", line 1091, in forward
    return self.base_model(
  File "/data/bwllm/venv/llama/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1511, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/data/bwllm/venv/llama/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1520, in _call_impl
